## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-15-18-44-15 +0000,nypost,Mayor Adams to land massive endorsements in ea...,https://nypost.com/2025/07/15/us-news/mayor-ad...
1,2025-07-15-18-44-15 +0000,nyt,Rising Inflation Underscores Risks in Trump’s ...,https://www.nytimes.com/2025/07/15/us/politics...
2,2025-07-15-18-32-59 +0000,bbc,Trump says Ukraine should not target Moscow,https://www.bbc.com/news/articles/cvg6zg6kelro
3,2025-07-15-18-27-07 +0000,nyt,Canada Wildfire Smoke Triggers Toronto and U.S...,https://www.nytimes.com/2025/07/14/world/canad...
4,2025-07-15-18-19-50 +0000,bbc,Adolescence star Owen Cooper becomes Emmys' yo...,https://www.bbc.com/news/articles/cjelpe2y3plo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2312/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,20
16,new,8
52,tariffs,6
74,china,5
17,tariff,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
1,2025-07-15-18-44-15 +0000,nyt,Rising Inflation Underscores Risks in Trump’s ...,https://www.nytimes.com/2025/07/15/us/politics...,39
44,2025-07-15-00-53-28 +0000,nypost,NATO chief warns Russia’s trading partners aft...,https://nypost.com/2025/07/14/us-news/nato-chi...,39
55,2025-07-14-19-51-43 +0000,nypost,Supreme Court lets Trump move forward with fir...,https://nypost.com/2025/07/14/us-news/supreme-...,37
22,2025-07-15-14-27-49 +0000,nypost,Trump announces ‘great’ trade deal with Indone...,https://nypost.com/2025/07/15/us-news/trump-an...,35
26,2025-07-15-13-32-31 +0000,nyt,China’s Economy Grows Steadily Despite Trump’s...,https://www.nytimes.com/2025/07/14/business/ch...,35


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
1,39,2025-07-15-18-44-15 +0000,nyt,Rising Inflation Underscores Risks in Trump’s ...,https://www.nytimes.com/2025/07/15/us/politics...
42,19,2025-07-15-01-25-19 +0000,nypost,Democratic city officers flocking to cop-frien...,https://nypost.com/2025/07/14/us-news/blue-cit...
6,17,2025-07-15-17-49-34 +0000,bbc,Carney says US-Canada trade deal likely to inc...,https://www.bbc.com/news/articles/c0q82qqxegno
10,17,2025-07-15-16-23-29 +0000,nypost,Largest teachers union slams ‘unlawful’ cuts t...,https://nypost.com/2025/07/15/us-news/largest-...
7,16,2025-07-15-17-35-23 +0000,nypost,Calif. Dem Rep. Ro Khanna’s bid to force relea...,https://nypost.com/2025/07/15/us-news/ro-khann...
40,16,2025-07-15-02-15-17 +0000,nypost,Three Democrats win double-digit support in ea...,https://nypost.com/2025/07/14/us-news/three-de...
51,16,2025-07-14-20-57-24 +0000,nypost,Sen. Rand Paul renews push for criminal probe ...,https://nypost.com/2025/07/14/us-news/sen-rand...
44,14,2025-07-15-00-53-28 +0000,nypost,NATO chief warns Russia’s trading partners aft...,https://nypost.com/2025/07/14/us-news/nato-chi...
32,14,2025-07-15-10-01-29 +0000,bbc,Watch: Flash flooding sweeps through New York ...,https://www.bbc.com/news/videos/c0l4259g3jdo
23,13,2025-07-15-14-26-07 +0000,nypost,Mother Jones editor ripped as ‘soulless ghoul’...,https://nypost.com/2025/07/15/us-news/mother-j...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
